# Heuristic Algorithm Pipeline with Example Implementation

In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import os
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession
import vertexai.preview.generative_models as generative_models
import json
from zoneinfo import ZoneInfo
import ast
import re
import calendar

In [2]:
def authenticate_and_initialize_services():
    SERVICE_ACCOUNT_FILE = 'my-project-353902-c63389c23751.json'
    SCOPES = [
        'https://www.googleapis.com/auth/calendar',
        'https://www.googleapis.com/auth/classroom.courses.readonly',
        'https://www.googleapis.com/auth/classroom.coursework.me.readonly'
    ]

    if not os.path.exists(SERVICE_ACCOUNT_FILE):
        print("Service account file does not exist.")
        return None, None

    try:
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        
        calendar_service = build('calendar', 'v3', credentials=creds)
        classroom_service = build('classroom', 'v1', credentials=creds)
        
        return calendar_service, classroom_service
    
    except Exception as e:
        print(f"An error occurred during authentication: {str(e)}")
        return None, None

# Usage
calendar_service, classroom_service = authenticate_and_initialize_services()

In [3]:
def get_free_time_slots(service, calendar_id, min_time, max_time):
    if isinstance(min_time, str):
        min_time = datetime.fromisoformat(min_time)
    if isinstance(max_time, str):
        max_time = datetime.fromisoformat(max_time)

    local_tz = ZoneInfo("America/Chicago")
    min_time = min_time.replace(tzinfo=local_tz)
    max_time = max_time.replace(tzinfo=local_tz)

    events_result = service.events().list(calendarId=calendar_id,
                                          timeMin=min_time.isoformat(),
                                          timeMax=max_time.isoformat(),
                                          singleEvents=True,
                                          orderBy='startTime').execute()
    events = events_result.get('items', [])
    
    free_slots = [(min_time, max_time)]

    for event in events:
        start = datetime.fromisoformat(event['start'].get('dateTime', event['start'].get('date')))
        end = datetime.fromisoformat(event['end'].get('dateTime', event['end'].get('date')))
        
        # Explicitly set the timezone and adjust for DST
        start = start.replace(tzinfo=local_tz)
        end = end.replace(tzinfo=local_tz)
        
        # Adjust for potential DST differences
        if start.dst() != min_time.dst():
            start += timedelta(hours=1)
        if end.dst() != min_time.dst():
            end += timedelta(hours=1)

        new_free_slots = []
        for free_start, free_end in free_slots:
            if end <= free_start or start >= free_end:
                new_free_slots.append((free_start, free_end))
            else:
                if start > free_start:
                    new_free_slots.append((free_start, start))
                if end < free_end:
                    new_free_slots.append((end, free_end))
        free_slots = new_free_slots

    return free_slots

chicago_tz = ZoneInfo("America/Chicago")
current_time = datetime.now(chicago_tz)

# Set the start time to the current time, rounded to the nearest minute
min_time = current_time.replace(second=0, microsecond=0)

# Set the end time to exactly 7 days from the current time
max_time = min_time + timedelta(days=7)


free_slots = get_free_time_slots(calendar_service, 'nickramen.uchicago@gmail.com', min_time, max_time)
print(free_slots)

[(datetime.datetime(2024, 8, 6, 12, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')), datetime.datetime(2024, 8, 6, 19, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago'))), (datetime.datetime(2024, 8, 6, 20, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')), datetime.datetime(2024, 8, 7, 8, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago'))), (datetime.datetime(2024, 8, 7, 11, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')), datetime.datetime(2024, 8, 7, 12, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago'))), (datetime.datetime(2024, 8, 7, 13, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')), datetime.datetime(2024, 8, 8, 9, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago'))), (datetime.datetime(2024, 8, 8, 12, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')), datetime.datetime(2024, 8, 9, 9, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago'))), (datetime.datetime(2024, 8, 9, 11, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')), datetime.datetime(2024, 8, 9,

In [4]:
def list_calendar_events(service, calendar_id, min_time, max_time):
    """
    List events from a Google Calendar within a specified time range.

    :param service: Authenticated Google Calendar API service object
    :param calendar_id: ID of the calendar to fetch events from
    :param min_time: Start of the time range (datetime or ISO format string)
    :param max_time: End of the time range (datetime or ISO format string)
    :return: List of event dictionaries
    """
    if isinstance(min_time, str):
        min_time = datetime.fromisoformat(min_time)
    if isinstance(max_time, str):
        max_time = datetime.fromisoformat(max_time)

    # Set timezone
    local_tz = ZoneInfo("America/Chicago")
    min_time = min_time.replace(tzinfo=local_tz)
    max_time = max_time.replace(tzinfo=local_tz)

    # Fetch events from the calendar
    events_result = service.events().list(calendarId=calendar_id,
                                          timeMin=min_time.isoformat(),
                                          timeMax=max_time.isoformat(),
                                          singleEvents=True,
                                          orderBy='startTime').execute()
    events = events_result.get('items', [])

    # Process and return the events
    processed_events = []
    for event in events:
        start = datetime.fromisoformat(event['start'].get('dateTime', event['start'].get('date')))
        end = datetime.fromisoformat(event['end'].get('dateTime', event['end'].get('date')))
        
        # Set timezone and adjust for DST
        start = start.replace(tzinfo=local_tz)
        end = end.replace(tzinfo=local_tz)
        
        processed_events.append({
            'summary': event.get('summary', 'No title'),
            'start': start,
            'end': end,
            'id': event['id']        })

    return processed_events

chicago_tz = ZoneInfo("America/Chicago")
current_time = datetime.now(chicago_tz)
min_time = current_time.replace(second=0, microsecond=0)
max_time = min_time + timedelta(days=7)

events = list_calendar_events(calendar_service, 'nickramen.uchicago@gmail.com', min_time, max_time)

In [5]:
events

[{'summary': 'Brunch with Mom',
  'start': datetime.datetime(2024, 8, 6, 10, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'end': datetime.datetime(2024, 8, 6, 12, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'id': 'k1qrmd6qtducatj0uvie75aj58_20240806T153000Z'},
 {'summary': 'Baseball Practice ',
  'start': datetime.datetime(2024, 8, 6, 19, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'end': datetime.datetime(2024, 8, 6, 20, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'id': '3ns042ibrshgo0vaj5d5ktblqu_20240807T003000Z'},
 {'summary': 'Therapy',
  'start': datetime.datetime(2024, 8, 7, 8, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'end': datetime.datetime(2024, 8, 7, 9, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'id': '3db8dik0kmqqcsht0ug5l284g3_20240807T133000Z'},
 {'summary': 'Class: Marketing Analytics',
  'start': datetime.datetime(2024, 8, 7, 9, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago')),
  'end': dateti

In [6]:
for start, end in free_slots:
    print(start)

2024-08-06 12:00:00-05:00
2024-08-06 20:30:00-05:00
2024-08-07 11:00:00-05:00
2024-08-07 13:30:00-05:00
2024-08-08 12:00:00-05:00
2024-08-09 11:00:00-05:00
2024-08-09 13:30:00-05:00
2024-08-12 11:00:00-05:00
2024-08-12 13:30:00-05:00


In [7]:
def parse_time_slots(free_slots):
    formatted_slots = {}
    
    for start, end in free_slots:
        current = start
        while current < end:
            day = current.strftime('%A')
            day_end = datetime.combine(current.date(), datetime.max.time()).replace(tzinfo=current.tzinfo)
            slot_end = min(end, day_end)
            
            # Round start_hour to nearest quarter hour
            start_hour = round((current.hour + current.minute / 60) * 4) / 4
            
            # Calculate duration
            duration = (slot_end - current).total_seconds() / 3600  # Duration in hours
            
            # Round duration to nearest quarter hour
            duration = round(duration * 4) / 4
            
            formatted_slots.setdefault(day, []).append((start_hour, duration))
            
            if slot_end == end:
                break
            current = slot_end.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
    
    # Adjust the first day if it doesn't start at midnight
    if formatted_slots:
        first_day = list(formatted_slots.keys())[0]
        first_slot = formatted_slots[first_day][0]
        if first_slot[0] > 0:
            formatted_slots[first_day] = [(first_slot[0], first_slot[1])]
    
    return formatted_slots

In [8]:
weekly_time_slots = parse_time_slots(free_slots)

In [9]:
def parse_study_plan(content):
    content = content.strip().lstrip('```python').rstrip('```').strip()
    try:
        # Attempt to directly evaluate the string as a Python literal
        return ast.literal_eval(content)
    except:
        return None

In [10]:
def parse_deadline(deadline_str, start_date):
    """
    Parse various deadline formats and return a datetime object.
    
    Supported formats:
    - "in X days"
    - "YYYY-MM-DD"
    - "Day at HH:MM AM/PM"
    - ISO 8601 format
    """
    # Check for "in X days" format
    in_days_match = re.match(r'in (\d+) days?', deadline_str, re.IGNORECASE)
    if in_days_match:
        days = int(in_days_match.group(1))
        return start_date + timedelta(days=days)
    
    # Check for YYYY-MM-DD format
    try:
        return datetime.strptime(deadline_str, "%Y-%m-%d")
    except ValueError:
        pass
    
    # Check for "Day at HH:MM AM/PM" format
    day_time_match = re.match(r'(\w+) at (\d{1,2}):(\d{2}) (AM|PM)', deadline_str, re.IGNORECASE)
    if day_time_match:
        day, hour, minute, ampm = day_time_match.groups()
        hour = int(hour)
        if ampm.upper() == 'PM' and hour != 12:
            hour += 12
        elif ampm.upper() == 'AM' and hour == 12:
            hour = 0
        target_date = start_date + timedelta(days=(list(calendar.day_name).index(day) - start_date.weekday() + 7) % 7)
        return target_date.replace(hour=hour, minute=int(minute), second=0, microsecond=0)
    
    # Check for ISO 8601 format
    try:
        return datetime.fromisoformat(deadline_str)
    except ValueError:
        pass
    
    # If no format matches, raise an error
    raise ValueError(f"Unsupported deadline format: {deadline_str}")

def get_enhanced_deadline_info(deadline_date, start_date):
    """Convert a deadline date to a more informative format."""
    days_until = (deadline_date - start_date).days
    weekday = deadline_date.strftime("%A")
    date_str = deadline_date.strftime("%Y-%m-%d")
    time_str = deadline_date.strftime("%I:%M %p")

    if days_until < 0:
        return f"Past due ({abs(days_until)} days ago) - {weekday}, {date_str} at {time_str}"
    elif days_until == 0:
        return f"Today ({weekday}) at {time_str}"
    elif days_until == 1:
        return f"Tomorrow ({weekday}) at {time_str}"
    elif days_until < 7:
        return f"In {days_until} days ({weekday}, {date_str}) at {time_str}"
    else:
        return f"In {days_until} days - {weekday}, {date_str} at {time_str}"

def prepare_enhanced_deadlines(deadlines, start_date):
    """Prepare deadlines with enhanced information."""
    enhanced_deadlines = {}
    for subject, deadline in deadlines.items():
        deadline_date = parse_deadline(deadline, start_date)
        enhanced_deadlines[subject] = get_enhanced_deadline_info(deadline_date, start_date)
    return enhanced_deadlines

# Example usage:
start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
deadlines = {
    "Math": "in 4 days",
    "English": "in 5 days",
    "Biology": "in 7 days",
    "Chemistry": "in 3 days"
}

enhanced_deadlines = prepare_enhanced_deadlines(deadlines, start_date)
print(enhanced_deadlines)

{'Math': 'In 4 days (Saturday, 2024-08-10) at 12:00 AM', 'English': 'In 5 days (Sunday, 2024-08-11) at 12:00 AM', 'Biology': 'In 7 days - Tuesday, 2024-08-13 at 12:00 AM', 'Chemistry': 'In 3 days (Friday, 2024-08-09) at 12:00 AM'}


In [11]:
def multiturn_generate_content(assignments, enhanced_deadlines, priorities, start_date):
    vertexai.init(project="adsp-capstone-convocrafters", location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-preview-0514")
    
    weekdays = [start_date + timedelta(days=i) for i in range(7)]
    weekdays_str = ", ".join([day.strftime("%A (%Y-%m-%d)") for day in weekdays])
    
    prompt = f"""
    Create a balanced study plan based on these assignments, deadlines, and priorities:

    Assignments and Durations:
    {assignments}

    Enhanced Deadlines:
    {enhanced_deadlines}

    Priorities (listed from highest to lowest):
    {priorities}

    Available Days (in order):
    {weekdays_str}

    Follow these constraints and guidelines:
    1. Use the provided durations exactly. The total study time for each subject must match the given duration.
    2. Spread the assignments across the available days based on deadlines and priorities.
    3. Prioritize subjects with imminent or past due deadlines. Allocate more time to these subjects earlier in the week.
    4. Only use the days provided in the Available Days list.
    5. For past due assignments, allocate time as soon as possible to catch up.
    6. Try to balance the workload across the week, avoiding excessive studying on any single day if possible.
    7. Consider both the priority order and deadlines when scheduling. Subjects listed earlier in the Priorities list are generally more important to the student.
    8. If a subject has less than 1 hour total duration, you may schedule it as a single session.
    9. Do not schedule any 0-hour study sessions. If a subject is included for a day, it must have a duration greater than 0.

    Output only the study plan as a Python dictionary with each available day as a key, and the value as a list of tuples.
    Each tuple should contain (subject, duration in hours, priority score).
    The priority score should be a number from 1 to 5, where 5 is the highest priority. This score should be based on a combination of:
    - The subject's position in the Priorities list
    - The urgency of the deadline
    - Any past due status

    Example format:
    {{
        {weekdays[0].strftime('%A (%Y-%m-%d)')}: [("Math", 2, 5), ("English", 1.5, 4)], 
        {weekdays[1].strftime('%A (%Y-%m-%d)')}: [("Biology", 2, 3), ("Math", 1, 5)]
    }}

    Ensure that the study plan is realistic, takes into account the urgency of deadlines, and never includes 0-hour study sessions.
    Subjects with higher priority scores should generally be given more favorable time slots or more frequent sessions.
    Do not include any explanations or additional code. Only output the study plan dictionary.
    """

    try:
        response = model.generate_content(prompt)
        content = response.text
        print("Raw response:", content)  # Debug print
        study_plan = parse_study_plan(content)
        if study_plan is None:
            raise ValueError("Unable to parse a valid study plan dictionary from the response")
        
        # Verify the study plan
        total_hours = {subject: sum(duration for day in study_plan.values() for s, duration, _ in day if s == subject)
                       for subject in assignments}
        
        if total_hours != assignments:
            raise ValueError(f"Study plan does not match the given estimates. Expected: {assignments}, Got: {total_hours}")
        
        # Check for 0-hour sessions
        for day, sessions in study_plan.items():
            if any(duration == 0 for _, duration, _ in sessions):
                raise ValueError(f"Study plan contains 0-hour sessions on {day}")
        
        return study_plan
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [12]:
assignments = {"Math": 8, "English": 5, "Biology": 2, "Chemistry": 4}
priorities = ["Math", "English", "Biology", "Chemistry"]
start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

weekly_study_plan = multiturn_generate_content(assignments, enhanced_deadlines, priorities, start_date)

Raw response: ```python
{
    'Tuesday (2024-08-06)': [('Math', 2, 5), ('Chemistry', 2, 4)],
    'Wednesday (2024-08-07)': [('Math', 2, 5), ('Chemistry', 2, 4)],
    'Thursday (2024-08-08)': [('Math', 2, 5), ('English', 2, 4)],
    'Friday (2024-08-09)': [('Math', 2, 5), ('English', 1, 4)],
    'Saturday (2024-08-10)': [('English', 2, 4)],
    'Sunday (2024-08-11)': [('Biology', 2, 3)],
    'Monday (2024-08-12)': []
}
```



In [13]:
weekly_study_plan

{'Tuesday (2024-08-06)': [('Math', 2, 5), ('Chemistry', 2, 4)],
 'Wednesday (2024-08-07)': [('Math', 2, 5), ('Chemistry', 2, 4)],
 'Thursday (2024-08-08)': [('Math', 2, 5), ('English', 2, 4)],
 'Friday (2024-08-09)': [('Math', 2, 5), ('English', 1, 4)],
 'Saturday (2024-08-10)': [('English', 2, 4)],
 'Sunday (2024-08-11)': [('Biology', 2, 3)],
 'Monday (2024-08-12)': []}

In [14]:
for day, sessions in weekly_study_plan.items():
        day_name = day.split()[0]
        print(day_name)

Tuesday
Wednesday
Thursday
Friday
Saturday
Sunday
Monday


In [15]:
def fit_study_times_into_timed_slots(weekly_plan, weekly_slots, start_hour, end_hour, buffer=0.5, morning_person=True):
    weekly_schedule = {}
    
    for day, sessions in weekly_plan.items():
        day_name = day.split()[0]
        
        sessions.sort(key=lambda x: x[2], reverse=True)
        daily_schedule = []
        time_slots = weekly_slots.get(day_name, [])
        
        adjusted_slots = [
            (max(start_time, start_hour), min(start_time + duration, end_hour))
            for start_time, duration in time_slots
        ]
        
        adjusted_slots.sort(key=lambda x: x[0], reverse=not morning_person)
        
        for subject, duration, priority in sessions:
            for i, (slot_start, slot_end) in enumerate(adjusted_slots):
                if slot_end - slot_start >= buffer:
                    study_start = slot_start + buffer if daily_schedule else slot_start
                    study_time = min(duration, slot_end - study_start)
                    study_end = study_start + study_time
                    
                    if study_end <= slot_end:
                        daily_schedule.append((subject, study_start, study_end))
                        duration -= study_time
                        adjusted_slots[i] = (study_end, slot_end)
                        
                        if duration == 0:
                            break
        
        if daily_schedule:
            weekly_schedule[day] = daily_schedule
    
    return weekly_schedule

In [16]:
scheduled_week = fit_study_times_into_timed_slots(
    weekly_study_plan, weekly_time_slots, start_hour=9, end_hour=20, buffer=0.50, morning_person=True)

In [17]:
scheduled_week

{'Tuesday (2024-08-06)': [('Math', 12.0, 14.0), ('Chemistry', 14.5, 16.5)],
 'Wednesday (2024-08-07)': [('Math', 11.0, 12.0),
  ('Math', 14.0, 15.0),
  ('Chemistry', 15.5, 17.5)],
 'Thursday (2024-08-08)': [('Math', 12.0, 14.0), ('English', 14.5, 16.5)],
 'Friday (2024-08-09)': [('Math', 9, 9.5),
  ('Math', 11.5, 12.0),
  ('Math', 14.0, 15.0),
  ('English', 15.5, 16.5)],
 'Saturday (2024-08-10)': [('English', 9, 11)],
 'Sunday (2024-08-11)': [('Biology', 9, 11)]}